<a href="https://colab.research.google.com/github/MiyoBran/Alura-ONE-G9/blob/main/formacion-Aprendiendo-a-hacer-ETL-G9-ONE/03-pandas-transformacion-manipulacion-datos/ETL_03_Pandas_Tratamiento_Manipulacion_Datos_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#**PROBLEMA DE NEGOCIO**


---




Se trata de un algoritmo capaz de analizar las características de una propiedad -como comodidades, tamaño, ocupación del espacio en un período determinado- y sugerir al anfitrión un precio a cobrar por tarifas diarias que garantice ganancias en momentos de alta demanda.

##**1. Entendiendo el Problema**


---




### 🧹 Data Wrangling: Ingesta de JSON Anidado

**El Problema Técnico (Datos Semi-Estructurados):**
A diferencia de un archivo CSV (que es inherentemente plano y tabular), el formato JSON soporta jerarquías profundas (diccionarios dentro de listas, dentro de diccionarios).

Al utilizar la función base `pd.read_json()`, Pandas mapea el primer nivel del JSON a columnas. Si los valores internos son también estructuras complejas, Pandas no las desglosa automáticamente; las inserta como objetos tipo `dict` dentro de la celda de la columna. Esto hace que los datos sean inutilizables para análisis matemático o estadístico directo, requiriendo un proceso posterior de **Normalización** (Flattening).

**Ejemplo del comportamiento por defecto:**
```python
import pandas as pd

# Lectura directa del archivo JSON crudo
ruta_archivo = '/content/datos_hosting.json'
datos = pd.read_json(ruta_archivo)

# Inspección inicial: Revela diccionarios anidados en las celdas
display(datos.head())
```

In [2]:
import pandas as pd

In [3]:
#Los datos necesitan ser normalizados
datos = pd.read_json('/content/drive/MyDrive/Pandas/datos_hosting.json')
datos.head()

,info_inmuebles
0,"{'evaluacion_general': '10.0', 'experiencia_lo..."
1,"{'evaluacion_general': '10.0', 'experiencia_lo..."
2,"{'evaluacion_general': '10.0', 'experiencia_lo..."
3,"{'evaluacion_general': '10.0', 'experiencia_lo..."
4,"{'evaluacion_general': '10.0', 'experiencia_lo..."


### 🗂️ Aplanamiento Estructural: El Método `json_normalize()`

**Concepto de Negocio (Integración de APIs):**
Cuando un sistema se conecta a la AFIP o a una pasarela de pagos (MercadoPago, Stripe), la respuesta rara vez es una tabla plana. Suele llegar como un gran objeto JSON con diccionarios anidados (ej. `{'cliente': {'nombre': 'Juan', 'id': 123}}`). Para que Pandas o una Base de Datos SQL puedan procesar esto, necesitamos transformar esa jerarquía en columnas estándar.

**La Solución Técnica:**
El método `pd.json_normalize()` es el estándar de la industria para el "aplanamiento" (Flattening) de diccionarios.
* **¿Qué hace?** Toma una columna entera que contiene diccionarios crudos y extrae cada "Llave" (Key) para convertirla en una nueva columna de nuestro DataFrame.
* **Limitación Arquitectónica:** Es excelente rompiendo diccionarios (`{}`), pero **no rompe listas** (`[]`). Si un valor dentro del diccionario era una lista de precios, la trasladará intacta a la nueva celda, requiriendo un paso posterior de limpieza (como `explode()`).

In [4]:
datos = pd.json_normalize(datos['info_inmuebles'])
datos.head()

,evaluacion_general,experiencia_local,max_hospedes,descripcion_local,descripcion_vecindad,cantidad_baños,cantidad_cuartos,cantidad_camas,modelo_cama,comodidades,cuota_deposito,cuota_limpieza,precio
0,10.0,--,1,[This clean and comfortable one bedroom sits r...,[Lower Queen Anne is near the Seattle Center (...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Real Bed, Futon, Futon, Pull-out Sofa, Real B...","[{Internet,""Wireless Internet"",Kitchen,""Free P...","[$0, $0, $0, $0, $0, $350.00, $350.00, $350.00...","[$0, $0, $0, $20.00, $15.00, $28.00, $35.00, $...","[$110.00, $45.00, $55.00, $52.00, $85.00, $50...."
1,10.0,--,10,[Welcome to the heart of the 'Ballard Brewery ...,"[--, Capital Hill is the heart of Seattle, bor...","[2, 3, 2, 3, 3, 3, 2, 1, 2, 2, 2]","[3, 4, 2, 3, 3, 3, 3, 3, 3, 4, 3]","[5, 6, 8, 3, 3, 5, 4, 5, 6, 7, 4]","[Real Bed, Real Bed, Real Bed, Real Bed, Real ...","[{TV,Internet,""Wireless Internet"",Kitchen,""Fre...","[$500.00, $300.00, $0, $300.00, $300.00, $360....","[$125.00, $100.00, $85.00, $110.00, $110.00, $...","[$350.00, $300.00, $425.00, $300.00, $285.00, ..."
2,10.0,--,11,[New modern house built in 2013. Spectacular ...,[Upper Queen Anne is a charming neighborhood f...,[4],[5],[7],[Real Bed],"[{TV,""Cable TV"",Internet,""Wireless Internet"",""...","[$1,000.00]",[$300.00],[$975.00]
3,10.0,--,12,[Our NW style home is 3200+ sq ft with 3 level...,[The Views from our top floor! Wallingford ha...,"[3, 3, 3, 3, 3, 3, 3, 3]","[6, 6, 5, 5, 5, 5, 4, 4]","[6, 6, 7, 8, 7, 7, 6, 6]","[Real Bed, Real Bed, Real Bed, Real Bed, Real ...","[{Internet,""Wireless Internet"",Kitchen,""Free P...","[$500.00, $500.00, $500.00, $500.00, $500.00, ...","[$225.00, $300.00, $250.00, $250.00, $250.00, ...","[$490.00, $550.00, $350.00, $350.00, $350.00, ..."
4,10.0,--,14,"[Perfect for groups. 2 bedrooms, full bathroom...",[Safeway grocery store within walking distance...,"[2, 3]","[2, 6]","[3, 9]","[Real Bed, Real Bed]","[{TV,Internet,""Wireless Internet"",Kitchen,""Fre...","[$300.00, $2,000.00]","[$40.00, $150.00]","[$200.00, $545.00]"


### 07-Desafío: trabajando en otros contextos

#### Trabajando en otros contextos

---

Ha llegado el momento de poner en práctica todo lo aprendido durante las clases. ¡Preparé dos proyectos adicionales para que los desarrollemos durante el curso, para asegurarnos de que podamos practicar mucho! Para ello trabajaremos con 2 nuevos conjuntos de datos, pero esta vez serán mucho más pequeños. Las bases de datos están disponibles para descargar aquí:


* [Proyecto Desafío 1: Ventas Online - dados_vendas_clientes.json](https://cdn3.gnarususercontent.com.br/2928-transformacao-manipulacao-dados/dados_vendas_clientes.json);
* [Proyecto Desafío 2: Administración de Condominios - dados_locacao_imoveis.json](https://cdn3.gnarususercontent.com.br/2928-transformacao-manipulacao-dados/dados_locacao_imoveis.json).

En cada clase desarrollaremos una etapa de los proyectos. Así que guarde su código de construcción para cada desafío para poder aplicarlo en los desafíos posteriores.

---

#### **Etapa 1**

* **Proyecto Desafío 1: Ventas Online**


El objetivo de este proyecto es analizar los resultados de un evento con los clientes de una empresa de venta online. Se recopiló un conjunto de datos que contiene los clientes que más gastaron en productos durante los 5 días de ventas, que es la duración del evento. Este análisis identificará al cliente con la mayor compra esta semana, quien recibirá un premio de la tienda, y posteriormente, puede ayudar a la empresa a crear nuevas estrategias para atraer más clientes.

La base de datos utilizada en este análisis es [dados_vendas_clientes.json](https://cdn3.gnarususercontent.com.br/2928-transformacao-manipulacao-dados/dados_vendas_clientes.json) y contiene información importante sobre los clientes, como el nombre registrado del cliente, el monto total pagado al momento de la compra y el día de la compra.

Sabiendo esta información, el desafío del proyecto 1: ventas online será abrir la base de datos con Pandas y aplicar [json_normalize](https://pandas.pydata.org/docs/reference/api/pandas.json_normalize.html).

---

* **Proyecto Desafío 2: Administración de Condominios**


Administrar condominios es una tarea que requiere mucha atención y organización. Entre las diversas responsabilidades de gestión se encuentra el cobro del alquiler a los inquilinos. Para garantizar la buena salud financiera de la empresa, es fundamental que estos pagos se realicen de forma regular y puntual. Sin embargo, sabemos que esto no siempre sucede.


Teniendo esto en cuenta, propongo un desafío de procesamiento de datos con el objetivo de analizar el retraso en el pago del alquiler en el condominio de algunos residentes. Pongo a disposición la base de datos [dados_locacao_imoveis](https://cdn3.gnarususercontent.com.br/2928-transformacao-manipulacao-dados/dados_locacao_imoveis.json).json, que contiene información sobre el departamento de los inquilinos, el día acordado para el pago del alquiler, el día en que se realiza el pago del alquiler y el monto del alquiler.


Con esta información, el desafío del proyecto 2: administración del condominio será similar al desafío del proyecto 1, abrir la base de datos con Pandas y aplicar [json_normalize](https://pandas.pydata.org/docs/reference/api/pandas.json_normalize.html). al DataFrame.

* **Proyecto Desafío 1: Ventas Online**



```Python
# Import de pandas
import pandas as pd
# Leer el archivo json con read_json
datos = pd.read_json('/content/dados_vendas_clientes.json')
# Aplicar json_normalize en la columna dados_vendas
datos = pd.json_normalize(datos['dados_vendas'])
# Mostrar valores
datos
```
* **Projecto Desafio 2: Administración de Condominios**


```Python
# Import de pandas
import pandas as pd
# Leer el archivo json con read_json
datos = pd.read_json('/content/dados_locacao_imoveis.json')
# Aplicar json_normalize en la columna dados_locacao
datos = pd.json_normalize(datos['dados_locacao'])
# Mostrar valores
datos
```




#**2. Datos numéricos**


---




In [5]:
columnas = list(datos.columns)
columnas

['evaluacion_general',
 'experiencia_local',
 'max_hospedes',
 'descripcion_local',
 'descripcion_vecindad',
 'cantidad_baños',
 'cantidad_cuartos',
 'cantidad_camas',
 'modelo_cama',
 'comodidades',
 'cuota_deposito',
 'cuota_limpieza',
 'precio']

### 📖 Diccionario de Datos (`datos_hosting.json`)

**Importancia Estructural:**
Antes de aplicar transformaciones o *Type Casting* (conversión de tipos), es obligatorio definir la naturaleza de cada variable. Esto guiará qué columnas deben ser numéricas (para cálculos estadísticos) y cuáles categóricas o de texto.

* **`evaluacion_general`**: Puntuación media otorgada al alojamiento. (Esperado: Numérico/Float).
* **`experiencia_local`**: Descripción de experiencias ofrecidas. (Esperado: Texto/String).
* **`max_hospedes`**: Capacidad máxima de personas. (Esperado: Numérico/Int).
* **`descripcion_local`**: Descripción general de la propiedad. (Esperado: Texto/String).
* **`descripcion_vecindad`**: Entorno o barrio de la propiedad. (Esperado: Texto/String).
* **`cantidad_baños`**: Baños disponibles. (Esperado: Numérico/Float o Int).
* **`cantidad_cuartos`**: Habitaciones disponibles. (Esperado: Numérico/Int).
* **`cantidad_camas`**: Camas disponibles. (Esperado: Numérico/Int).
* **`modelo_cama`**: Clasificación del tipo de cama. (Esperado: Categórico/Texto).
* **`comodidades`**: Lista de servicios/amenities. (Esperado: Colección/Texto).
* **`cuota_deposito`**: Tarifa de seguridad mínima. (Esperado: Numérico/Float).
* **`cuota_limpieza`**: Cargo por servicio de limpieza. (Esperado: Numérico/Float).
* **`precio`**: Tarifa base diaria. (Esperado: Numérico/Float).

### 💣 Desanidamiento de Listas: El Método `explode()`

**El Problema Técnico (Falta de Granularidad):**
Cuando consumimos APIs o archivos JSON, es común que propiedades múltiples (como "comodidades" o "precios históricos") vengan empaquetadas en forma de listas de Python `[valor1, valor2, valor3]` dentro de una sola celda del DataFrame. Los motores de bases de datos y las funciones estadísticas requieren valores **escalares** (un solo dato por celda).

**La Solución de la Industria (`explode`):**
El método `explode()` toma una columna que contiene listas y "explota" sus elementos. Por cada elemento dentro de la lista, crea una nueva fila exacta en el DataFrame, copiando los valores de las demás columnas y manteniendo el Índice original para saber que todos esos datos provenían del mismo registro.

**Ejemplo Teórico Aislado:**
Imaginemos una factura con múltiples precios empaquetados.

```python
import pandas as pd

# 1. Recreamos el problema (Datos anidados en listas)
datos_dummy = pd.DataFrame({
    'ID_Factura': ['F-001', 'F-002'],
    'Cliente': ['Empresa A', 'Empresa B'],
    'Precios_Items': [[150, 200], [500, 50, 120]] # <--- El problema
})

print("❌ DataFrame Original (Precios atrapados en listas):")
display(datos_dummy)

# 2. Aplicamos la solución arquitectónica
# Usamos explode indicando qué columna queremos reventar
datos_granulares = datos_dummy.explode('Precios_Items')

print("\n✅ DataFrame Granular (Un precio por fila, listos para sumar/promediar):")
display(datos_granulares)
```
Se puede ejecutar el codigo anterior en una celda de codigo temporal para ver y entender el ejemplo

In [6]:
# cuando visualizamos los datos previamente, vimos que desde la columna 4 (pos
# 3) en adelante es necesario hacer el explode , ya que son todas listas.
datos = datos.explode(columnas[3:])
datos.head()

,evaluacion_general,experiencia_local,max_hospedes,descripcion_local,descripcion_vecindad,cantidad_baños,cantidad_cuartos,cantidad_camas,modelo_cama,comodidades,cuota_deposito,cuota_limpieza,precio
0,10.0,--,1,This clean and comfortable one bedroom sits ri...,Lower Queen Anne is near the Seattle Center (s...,1,1,1,Real Bed,"{Internet,""Wireless Internet"",Kitchen,""Free Pa...",$0,$0,$110.00
0,10.0,--,1,Our century old Upper Queen Anne house is loca...,"Upper Queen Anne is a really pleasant, unique ...",1,1,1,Futon,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",$0,$0,$45.00
0,10.0,--,1,Cozy room in two-bedroom apartment along the l...,The convenience of being in Seattle but on the...,1,1,1,Futon,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",$0,$0,$55.00
0,10.0,--,1,Very lovely and cozy room for one. Convenientl...,"Ballard is lovely, vibrant and one of the most...",1,1,1,Pull-out Sofa,"{Internet,""Wireless Internet"",Kitchen,""Free Pa...",$0,$20.00,$52.00
0,10.0,--,1,The “Studio at Mibbett Hollow' is in a Beautif...,--,1,1,1,Real Bed,"{""Wireless Internet"",Kitchen,""Free Parking on ...",$0,$15.00,$85.00


In [7]:
# en el paso anterior todos los registros que fueron "explotados" aohra tienen
# el mismo indice ya que fue dividido el registro original en muchos, y para
# solucionar esto usamos el reset de los mismos.
datos.reset_index(inplace = True, drop = True)
datos.head()

,evaluacion_general,experiencia_local,max_hospedes,descripcion_local,descripcion_vecindad,cantidad_baños,cantidad_cuartos,cantidad_camas,modelo_cama,comodidades,cuota_deposito,cuota_limpieza,precio
0,10.0,--,1,This clean and comfortable one bedroom sits ri...,Lower Queen Anne is near the Seattle Center (s...,1,1,1,Real Bed,"{Internet,""Wireless Internet"",Kitchen,""Free Pa...",$0,$0,$110.00
1,10.0,--,1,Our century old Upper Queen Anne house is loca...,"Upper Queen Anne is a really pleasant, unique ...",1,1,1,Futon,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",$0,$0,$45.00
2,10.0,--,1,Cozy room in two-bedroom apartment along the l...,The convenience of being in Seattle but on the...,1,1,1,Futon,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",$0,$0,$55.00
3,10.0,--,1,Very lovely and cozy room for one. Convenientl...,"Ballard is lovely, vibrant and one of the most...",1,1,1,Pull-out Sofa,"{Internet,""Wireless Internet"",Kitchen,""Free Pa...",$0,$20.00,$52.00
4,10.0,--,1,The “Studio at Mibbett Hollow' is in a Beautif...,--,1,1,1,Real Bed,"{""Wireless Internet"",Kitchen,""Free Parking on ...",$0,$15.00,$85.00


In [8]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   evaluacion_general    3818 non-null   object
 1   experiencia_local     3818 non-null   object
 2   max_hospedes          3818 non-null   object
 3   descripcion_local     3818 non-null   object
 4   descripcion_vecindad  3818 non-null   object
 5   cantidad_baños        3818 non-null   object
 6   cantidad_cuartos      3818 non-null   object
 7   cantidad_camas        3818 non-null   object
 8   modelo_cama           3818 non-null   object
 9   comodidades           3818 non-null   object
 10  cuota_deposito        3818 non-null   object
 11  cuota_limpieza        3818 non-null   object
 12  precio                3818 non-null   object
dtypes: object(13)
memory usage: 387.9+ KB


### 🚨 Diagnóstico de Tipos de Datos (`Dtype: object`)

**El Problema Técnico (Rendimiento y Vectorización):**
Tras ejecutar `datos.info()`, observamos que las 13 columnas tienen el tipo de dato `object`. En Pandas, `object` es un tipo de dato genérico que generalmente representa cadenas de texto (Strings) o estructuras mixtas (punteros a bloques de memoria dispersos).

**Consecuencias de no realizar *Type Casting* (Conversión):**
1. **Fallo en Operaciones Matemáticas:** No podemos aplicar funciones estadísticas (como `mean()`, `sum()`) sobre columnas como `precio` o `cantidad_baños` porque el motor de Python las interpretará como concatenación de texto o arrojará un `TypeError`.
2. **Consumo de Memoria:** El tipo `object` consume drásticamente más memoria RAM que los tipos numéricos nativos de C (como `int64` o `float64`), ya que no utiliza arreglos contiguos en memoria.

**Próximo Paso Lógico (Data Cleaning):**
Debemos inspeccionar las columnas numéricas para identificar caracteres especiales (como `$`, `,` o `--`) que están forzando a Pandas a clasificarlas como texto, limpiar esos caracteres y aplicar métodos de conversión (como `astype()`).

In [9]:
import numpy as np

In [10]:
datos['max_hospedes'] = datos['max_hospedes'].astype(np.int64)

In [11]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   evaluacion_general    3818 non-null   object
 1   experiencia_local     3818 non-null   object
 2   max_hospedes          3818 non-null   int64 
 3   descripcion_local     3818 non-null   object
 4   descripcion_vecindad  3818 non-null   object
 5   cantidad_baños        3818 non-null   object
 6   cantidad_cuartos      3818 non-null   object
 7   cantidad_camas        3818 non-null   object
 8   modelo_cama           3818 non-null   object
 9   comodidades           3818 non-null   object
 10  cuota_deposito        3818 non-null   object
 11  cuota_limpieza        3818 non-null   object
 12  precio                3818 non-null   object
dtypes: int64(1), object(12)
memory usage: 387.9+ KB


In [12]:
col_numericas = ['cantidad_baños', 'cantidad_cuartos', 'cantidad_camas']

In [13]:
datos[col_numericas] = datos[col_numericas].astype(np.int64)

In [14]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   evaluacion_general    3818 non-null   object
 1   experiencia_local     3818 non-null   object
 2   max_hospedes          3818 non-null   int64 
 3   descripcion_local     3818 non-null   object
 4   descripcion_vecindad  3818 non-null   object
 5   cantidad_baños        3818 non-null   int64 
 6   cantidad_cuartos      3818 non-null   int64 
 7   cantidad_camas        3818 non-null   int64 
 8   modelo_cama           3818 non-null   object
 9   comodidades           3818 non-null   object
 10  cuota_deposito        3818 non-null   object
 11  cuota_limpieza        3818 non-null   object
 12  precio                3818 non-null   object
dtypes: int64(4), object(9)
memory usage: 387.9+ KB


In [15]:
datos['evaluacion_general'] = datos['evaluacion_general'].astype(np.float64)

In [16]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   evaluacion_general    3162 non-null   float64
 1   experiencia_local     3818 non-null   object 
 2   max_hospedes          3818 non-null   int64  
 3   descripcion_local     3818 non-null   object 
 4   descripcion_vecindad  3818 non-null   object 
 5   cantidad_baños        3818 non-null   int64  
 6   cantidad_cuartos      3818 non-null   int64  
 7   cantidad_camas        3818 non-null   int64  
 8   modelo_cama           3818 non-null   object 
 9   comodidades           3818 non-null   object 
 10  cuota_deposito        3818 non-null   object 
 11  cuota_limpieza        3818 non-null   object 
 12  precio                3818 non-null   object 
dtypes: float64(1), int64(4), object(8)
memory usage: 387.9+ KB


### 🔄 Transformación Estructural: *Type Casting* (Conversión de Tipos)

**El Proceso de Limpieza (Data Cleaning):**
Para que los datos sean utilizables analíticamente, debemos forzar su conversión del tipo genérico `object` a tipos de datos optimizados de la librería **NumPy** (`np.int64` para enteros y `np.float64` para decimales).

**El Comportamiento de `astype()`:**
* **Caso de Éxito (Columnas Limpias):** Si la columna solo contiene números en formato texto (ej. `"4"`), `astype(np.int64)` la convierte sin problemas, reduciendo el consumo de memoria y habilitando operaciones matemáticas.
* **Manejo de Nulos (El caso `--`):** Al convertir la columna `evaluacion_general` a `float64`, Pandas tradujo automáticamente los caracteres extraños `--` a valores nulos verdaderos (`NaN`). Por ello, el conteo de registros no nulos bajó de 3818 a 3162.
* **Caso de Fallo (Caracteres Críticos):** Columnas como `precio` contienen símbolos monetarios (`$`) y comas separadoras de miles (`,`). `astype()` fallará con un `ValueError` al intentar convertir "$1,000.00" a float. **Requieren una limpieza de Strings previa a la conversión.**

### 💾 Arquitectura de Memoria: Precisión de Tipos Numéricos

**Concepto Gerencial (Optimización vs. Riesgo):**
Por defecto, Pandas utiliza una precisión de 64 bits (`int64` y `float64`), lo cual es computacionalmente seguro pero costoso en memoria RAM. Optimizar implica reducir (Downcasting) el tipo de dato para que ocupe menos bytes, evaluando estrictamente los límites máximos permitidos por la regla de negocio para evitar un fallo crítico (Overflow).

**Matriz de Capacidad para Enteros (Números sin decimales):**

| Tipo de Dato (NumPy) | Consumo en RAM | Límite Mínimo | Límite Máximo | Caso de Uso Empresarial |
| :--- | :--- | :--- | :--- | :--- |
| `np.int8` | 1 Byte (8 bits) | -128 | 127 | Edades de clientes, cantidad de cuartos. |
| `np.int16` | 2 Bytes (16 bits) | -32.768 | 32.767 | Año de facturación, cantidad de empleados. |
| `np.int32` | 4 Bytes (32 bits) | -2.147.483.648 | 2.147.483.647 | IDs de facturas, Códigos de producto. |
| `np.int64` | 8 Bytes (64 bits) | -9.223 Trillones | 9.223 Trillones | Big Data, IDs globales únicos (UUID). |

**Precisión en Punto Flotante (Números Decimales):**
* **`np.float64` (Doble Precisión):** Ocupa 8 bytes. Soporta hasta 15 dígitos decimales. Obligatorio para cálculos financieros precisos, coordenadas GPS exactas o variables científicas.
* **`np.float32` (Precisión Simple):** Ocupa 4 bytes. Soporta hasta 7 dígitos decimales. Útil para promedios de encuestas o porcentajes estadísticos menores.

In [17]:
#creamos nuestra funcion lambda que limpia
datos['precio'] = datos['precio'].apply(lambda x: x.replace('$','').replace(',','').strip())

In [18]:
datos['precio'] = datos['precio'].astype(np.float64)

In [19]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   evaluacion_general    3162 non-null   float64
 1   experiencia_local     3818 non-null   object 
 2   max_hospedes          3818 non-null   int64  
 3   descripcion_local     3818 non-null   object 
 4   descripcion_vecindad  3818 non-null   object 
 5   cantidad_baños        3818 non-null   int64  
 6   cantidad_cuartos      3818 non-null   int64  
 7   cantidad_camas        3818 non-null   int64  
 8   modelo_cama           3818 non-null   object 
 9   comodidades           3818 non-null   object 
 10  cuota_deposito        3818 non-null   object 
 11  cuota_limpieza        3818 non-null   object 
 12  precio                3818 non-null   float64
dtypes: float64(2), int64(4), object(7)
memory usage: 387.9+ KB


### 🧹 Sanitización Financiera: Limpieza de Caracteres Especiales

**El Desafío de Negocio:**
Los sistemas de extracción de datos (Web Scraping o APIs) suelen traer los precios formateados para lectura humana (ej. `$1,250.00`). Para que un motor de base de datos o Pandas pueda realizar cálculos matemáticos, debemos eliminar los símbolos monetarios y los separadores de miles, dejando solo el número puro y el punto decimal.

**La Solución Estándar (Vectorización con `.str`):**
En lugar de usar `apply(lambda)` que procesa fila por fila de forma lenta, utilizamos el "Acelerador de Strings" de Pandas (`.str`). Esto aplica las transformaciones a toda la columna simultáneamente (Vectorización).

**Pasos de la Sanitización en Cascada:**
1. `.str.replace('$', '')`: Elimina el símbolo de la moneda.
2. `.str.replace(',', '')`: Elimina la coma separadora de miles (vital para que el número no se fragmente).
3. `.str.strip()`: Elimina espacios en blanco accidentales al principio o al final de la celda.
4. `.astype(np.float64)`: Una vez que el texto está limpio, finalmente lo convertimos a formato decimal matemático.


```Python
import numpy as np

# 1. Inspección previa (Vemos la "basura" visual)
print("Antes de la limpieza:")
display(datos['precio'].head(3))

# 2. Aplicamos la limpieza vectorizada (Clean Code y Alto Rendimiento)
# Enlazamos los métodos de string uno tras otro
datos['precio'] = datos['precio'].str.replace('$', '', regex=False) \
                                 .str.replace(',', '', regex=False) \
                                 .str.strip()

# 3. Conversión al tipo de dato correcto (Type Casting)
datos['precio'] = datos['precio'].astype(np.float64)

# 4. Auditoría final
print("\nDespués de la limpieza y conversión:")
display(datos['precio'].head(3))
datos.info()
```



In [20]:
datos[['cuota_deposito', 'cuota_limpieza']] = datos[['cuota_deposito', 'cuota_limpieza']].applymap(lambda x: x.replace('$','').replace(',','').strip())

/tmp/ipython-input-2696303735.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  datos[['cuota_deposito', 'cuota_limpieza']] = datos[['cuota_deposito', 'cuota_limpieza']].applymap(lambda x: x.replace('$','').replace(',','').strip())


In [21]:
datos[['cuota_deposito', 'cuota_limpieza']] = datos[['cuota_deposito', 'cuota_limpieza']].astype(np.float64)

In [22]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   evaluacion_general    3162 non-null   float64
 1   experiencia_local     3818 non-null   object 
 2   max_hospedes          3818 non-null   int64  
 3   descripcion_local     3818 non-null   object 
 4   descripcion_vecindad  3818 non-null   object 
 5   cantidad_baños        3818 non-null   int64  
 6   cantidad_cuartos      3818 non-null   int64  
 7   cantidad_camas        3818 non-null   int64  
 8   modelo_cama           3818 non-null   object 
 9   comodidades           3818 non-null   object 
 10  cuota_deposito        3818 non-null   float64
 11  cuota_limpieza        3818 non-null   float64
 12  precio                3818 non-null   float64
dtypes: float64(4), int64(4), object(5)
memory usage: 387.9+ KB


### 🧹 Limpieza Masiva de Columnas: El Método `applymap()`

**El Desafío Técnico:**
Teníamos múltiples columnas financieras (`cuota_deposito` y `cuota_limpieza`) infectadas con caracteres especiales (`$`, `,`). Limpiarlas una por una genera código repetitivo (rompe el principio *DRY - Don't Repeat Yourself*).

**La Solución Estructural (`applymap`):**
En lugar de procesar una sola Serie (columna), filtramos el DataFrame para aislar todas las columnas financieras. Luego usamos `applymap()`, que recorre cada celda individual de esa matriz bidimensional y le aplica la función lambda de sanitización (borrar `$`, borrar `,` y eliminar espacios).

**Código Implementado (Flujo de Trabajo):**
1. **Selección Múltiple:** `datos[['col1', 'col2']]`
2. **Transformación Masiva:** `.applymap(lambda x: x.replace('$','').replace(',','').strip())`
3. **Type Casting Masivo:** `.astype(np.float64)`

*(Nota de Arquitectura: En versiones de Pandas 2.1+, el método `applymap()` ha sido renombrado a `map()` para mayor claridad semántica).*

### 📐 Arquitectura Dimensional: Series vs DataFrames (`apply` vs `map`)

**Concepto Estructural:**
El error más común en la limpieza de datos con Pandas surge de no identificar la dimensionalidad del objeto que estamos manipulando.

1. **`pd.Series` (Unidimensional - 1D):**
   * **Sintaxis:** Un solo corchete `datos['precio']`.
   * **Comportamiento:** Representa un vector (una sola columna).
   * **Método de Iteración:** `.apply(lambda x: ...)`. Aplica la función `x` a cada valor escalar (celda) de la columna, de arriba hacia abajo.

2. **`pd.DataFrame` (Bidimensional - 2D):**
   * **Sintaxis:** Doble corchete `datos[['cuota_deposito', 'cuota_limpieza']]`.
   * **Comportamiento:** Representa una matriz o sub-tabla (múltiples columnas).
   * **Método de Iteración:** `.applymap(lambda x: ...)` (o `.map()` en Pandas 2.1+). Diseñado específicamente para recorrer la cuadrícula bidimensional, aplicando la función `x` a cada celda individual de todas las columnas seleccionadas.
   * **El Error Común:** Intentar usar `.apply()` con una función escalar sobre un DataFrame generará errores como `AttributeError`, ya que `.apply()` intentará pasar una Serie completa a la función `lambda`, en lugar de un valor de celda.

### 09 Desafío: trabajando en otros contextos


Pongamos nuevamente en práctica todo lo que aprendimos durante la clase. He puesto los 2 nuevos conjuntos de datos disponibles para descargar a continuación:

* Proyecto Desafío 1: Ventas Online - [dados_vendas_clientes.json](https://cdn3.gnarususercontent.com.br/2928-transformacao-manipulacao-dados/dados_vendas_clientes.json);
* Proyecto Desafío 2: Administración de Condominios - [dados_locacao_imoveis.json](https://cdn3.gnarususercontent.com.br/2928-transformacao-manipulacao-dados/dados_locacao_imoveis.json).

Recuerda: Hay dos proyectos de tratamiento que se construirán durante el curso. Así que guarde su código de construcción para cada desafío para poder aplicarlo a desafíos posteriores.

Etapa 2

* Proyecto Desafío 1: Ventas Online

Leímos la base de datos en el desafío anterior, ahora podemos seguir adelante con la transformación de estos datos. Así, el nuevo desafío del proyecto 1 será dividido en algunas metas:

* Eliminar datos en listas dentro del DataFrame;
* Verificar tipos de datos;
* Identificar columnas numéricas;
* Transformar la columna numérica a tipo numérico.
---

* Proyecto Desafío 2: Administración de Condominios

Leímos la base de datos en el desafío anterior, ahora podemos seguir adelante con la transformación de estos datos. Entonces, de la misma manera que en el proyecto 1, el desafío del proyecto 2 está listado en algunas metas:

* Eliminar datos en listas dentro del DataFrame;
* Verificar tipos de datos;
* Identificar columnas numéricas;
* Transformar la columna numérica a tipo numérico.

Proyecto Desafío 1: Ventas Online


```
# Colectar los valores de las columnas y verificar
columnas = list(datos.columns)
columnas

# Destrincar las listas con explode
datos = datos.explode(columnas[1:])
# Resetear los index de las líneas
datos.reset_index(drop=True,inplace=True)
# Observar el DataFrame
datos

# Verificar los tipos de datos con info
datos.info()

# La columna numérica es el 'Valor da compra'
datos['Valor da compra']

# Iniciar la transformación
# Import de la biblioteca numpy
import numpy as np
# Remover los textos presentes en la base
# Cambiar las comas separadoras del decimal por punto
datos['Valor da compra'] = datos['Valor da compra'].apply(lambda x: x.replace('R$ ', '').replace(',','.').strip())
# Cambiar los tipo de datos para float
datos['Valor da compra'] = datos['Valor da compra'].astype(np.float64)
# Verificar la transformación
datos.info()
```



Projecto Desafio 2: Administración de Condominios



```
# Colectar los valores de las columnas y verificar
columnas = list(datos.columns)
columnas

# Destrincar las listas con explode
datos = datos.explode(columnas[1:])
# Resetear los index de las líneas
datos.reset_index(drop=True,inplace=True)
# Observar el DataFrame
datos

# Verificar los tipos de datos con info
datos.info()

# La columna numérica es el 'valor_aluguel'
datos['valor_aluguel']

# Iniciar la transformación
# Import de la biblioteca numpy
import numpy as np
# Remover los textos presentes en la base
# Cambiar las comas separadoras del decimal por punto
datos['valor_aluguel'] = datos['valor_aluguel'].apply(lambda x: x.replace('$ ', '').replace(' reais', '').replace(',','.').strip())
# Cambiar los tipos de datos para float
datos['valor_aluguel'] = datos['valor_aluguel'].astype(np.float64)
# Verificar la transformación
datos.info()
```



#**3. Datos de texto**


---




In [23]:
datos.head()

,evaluacion_general,experiencia_local,max_hospedes,descripcion_local,descripcion_vecindad,cantidad_baños,cantidad_cuartos,cantidad_camas,modelo_cama,comodidades,cuota_deposito,cuota_limpieza,precio
0,10.0,--,1,This clean and comfortable one bedroom sits ri...,Lower Queen Anne is near the Seattle Center (s...,1,1,1,Real Bed,"{Internet,""Wireless Internet"",Kitchen,""Free Pa...",0.0,0.0,110.0
1,10.0,--,1,Our century old Upper Queen Anne house is loca...,"Upper Queen Anne is a really pleasant, unique ...",1,1,1,Futon,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",0.0,0.0,45.0
2,10.0,--,1,Cozy room in two-bedroom apartment along the l...,The convenience of being in Seattle but on the...,1,1,1,Futon,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",0.0,0.0,55.0
3,10.0,--,1,Very lovely and cozy room for one. Convenientl...,"Ballard is lovely, vibrant and one of the most...",1,1,1,Pull-out Sofa,"{Internet,""Wireless Internet"",Kitchen,""Free Pa...",0.0,20.0,52.0
4,10.0,--,1,The “Studio at Mibbett Hollow' is in a Beautif...,--,1,1,1,Real Bed,"{""Wireless Internet"",Kitchen,""Free Parking on ...",0.0,15.0,85.0


In [24]:
datos['descripcion_local'] = datos['descripcion_local'].str.lower()

In [25]:
datos.head()

,evaluacion_general,experiencia_local,max_hospedes,descripcion_local,descripcion_vecindad,cantidad_baños,cantidad_cuartos,cantidad_camas,modelo_cama,comodidades,cuota_deposito,cuota_limpieza,precio
0,10.0,--,1,this clean and comfortable one bedroom sits ri...,Lower Queen Anne is near the Seattle Center (s...,1,1,1,Real Bed,"{Internet,""Wireless Internet"",Kitchen,""Free Pa...",0.0,0.0,110.0
1,10.0,--,1,our century old upper queen anne house is loca...,"Upper Queen Anne is a really pleasant, unique ...",1,1,1,Futon,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",0.0,0.0,45.0
2,10.0,--,1,cozy room in two-bedroom apartment along the l...,The convenience of being in Seattle but on the...,1,1,1,Futon,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",0.0,0.0,55.0
3,10.0,--,1,very lovely and cozy room for one. convenientl...,"Ballard is lovely, vibrant and one of the most...",1,1,1,Pull-out Sofa,"{Internet,""Wireless Internet"",Kitchen,""Free Pa...",0.0,20.0,52.0
4,10.0,--,1,the “studio at mibbett hollow' is in a beautif...,--,1,1,1,Real Bed,"{""Wireless Internet"",Kitchen,""Free Parking on ...",0.0,15.0,85.0


### 📝 Procesamiento de Lenguaje Natural (NLP): Normalización y Tokenización

**El Valor de Negocio (Minería de Texto):**
Las variables de texto libre (como `descripcion_local`) esconden un inmenso valor predictivo. Palabras como "lujoso", "céntrico" o "ruidoso" impactan directamente en el precio de un servicio o en la satisfacción de un cliente. Para que un algoritmo de Machine Learning pueda ponderar estas palabras, primero debemos estructurar el texto.

**Fase 1: Normalización (Minúsculas)**
Los lenguajes de programación son *Case Sensitive* (sensibles a mayúsculas). Para el motor analítico, "Casa", "CASA" y "casa" son tres entidades completamente distintas.
El primer paso de limpieza es la homogenización mediante el método vectorizado `.str.lower()`, el cual convierte todos los caracteres del DataFrame a minúsculas, unificando el vocabulario.

**Fase 2: Tokenización (Concepto Teórico)**
Es el proceso de dividir una cadena de texto larga en unidades granulares más pequeñas llamadas "tokens" (generalmente palabras individuales o frases cortas). Esto permite contar la frecuencia de aparición de cada token y asignarle un peso matemático.

***Alternativa:***

```
df['experiencias_clientes'] = df['experiencias_clientes'].apply(lambda x: x.upper())
```



Se usa apply para aplicar una función a cada elemento de una columna. En este caso, la función lambda se utiliza para transformar todas las letras de la columna experiencias_clientes a mayúsculas. Al usar la función lambda dentro del método apply, estamos aplicando la función elemento por elemento en la columna.

In [26]:
datos['descripcion_local'][3169]

"built, run and supported by seattle tech and start up veterans, grokhome's focus is to create a supportive environment for smart people working on interesting projects, start ups and more. this listing is an upper bunk, in a 2-person shared room. *note: this fall, there will be major renovations happening on one kitchen and bathroom at a time. there will always be two other working kitchens and two working bathrooms in the house. we'll work to minimize the impact these renovations have on your stay. **this listing is only available to those working in the tech/science space. live in a hacker house, and immerse yourself in the seattle tech scene. you can expect to be surrounded by smart people solving big problems or working on something fun. we have frequent demo nights, and love when our guests share something they are passionate about. if you're new to the city, our deep ties to the seattle tech scene can help you get involved. expand your network, develop your ideas, and learn some

In [27]:
#utilizamos regex para
datos['descripcion_local'] = datos['descripcion_local'].str.replace('[^a-zA-Z0-9\-\']',' ',regex=True)

<>:2: SyntaxWarning: invalid escape sequence '\-'
<>:2: SyntaxWarning: invalid escape sequence '\-'
/tmp/ipython-input-1750333583.py:2: SyntaxWarning: invalid escape sequence '\-'
  datos['descripcion_local'] = datos['descripcion_local'].str.replace('[^a-zA-Z0-9\-\']',' ',regex=True)


In [28]:
datos['descripcion_local'] = datos['descripcion_local'].str.replace('(?<!\w)-(?!\w)',' ',regex=True)

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
/tmp/ipython-input-177244694.py:1: SyntaxWarning: invalid escape sequence '\w'
  datos['descripcion_local'] = datos['descripcion_local'].str.replace('(?<!\w)-(?!\w)',' ',regex=True)


In [29]:
datos['descripcion_local'].head()

,descripcion_local
0,this clean and comfortable one bedroom sits ri...
1,our century old upper queen anne house is loca...
2,cozy room in two-bedroom apartment along the l...
3,very lovely and cozy room for one convenientl...
4,the studio at mibbett hollow' is in a beautif...


### 🧹 NLP Fase 2: Limpieza Quirúrgica con Regex (Expresiones Regulares)

**El Problema de los Caracteres Especiales:**
Los textos libres contienen "ruido" (signos de exclamación, asteriscos, emojis) que confunden a los algoritmos de Machine Learning. Debemos limpiar este ruido conservando la estructura gramatical útil (como palabras compuestas: `wi-fi`).

**La Estrategia Regex (Búsqueda por Patrones):**
Utilizamos el parámetro `regex=True` dentro del método `.str.replace()` para indicarle a Pandas que no busque una palabra literal, sino un patrón lógico.

**Paso 1: Filtro de Negación (`[^...]`)**
* **Patrón:** `(?i)[^a-z0-9\-\']`
* **Explicación:** El símbolo `^` al principio de los corchetes significa **NEGACIÓN**. Le instruimos a Pandas: "Reemplaza cualquier carácter que **NO** sea una letra (`a-z`), un número (`0-9`), un guión literal (`\-`) o una comilla simple (`\'`)".

**Paso 2: Filtro de Guiones Huérfanos (`(?<!\w)-(?!\w)`)**
* **El Problema:** El paso 1 salvó los guiones para conservar palabras como `pet-friendly`. Pero dejó vivos guiones decorativos (` - `).
* **El Patrón Lookaround:** `(?<!\w)-(?!\w)` busca un guión (`-`) PERO verifica que no tenga una letra/número detrás (`(?<!\w)`) ni adelante (`(?!\w)`). Si el guión está aislado, lo destruye.



```Python
# ==============================================================================
# FASE 2: LIMPIEZA REGEX (Descripción Local)
# ==============================================================================

print("❌ Texto Original (Con ruido):")
display(datos['descripcion_local'].iloc[3169][:150]) # Vemos los primeros 150 caracteres del registro problemático

# PASO 1: Eliminar todo excepto letras, números, guiones y comillas simples
# Nota: (?i) hace que el regex ignore mayúsculas/minúsculas por seguridad
datos['descripcion_local'] = datos['descripcion_local'].str.replace(r'(?i)[^a-z0-9\-\']', ' ', regex=True)

# PASO 2: Eliminar guiones "huérfanos" (que no conectan palabras)
# \w significa "cualquier carácter alfanumérico".
datos['descripcion_local'] = datos['descripcion_local'].str.replace(r'(?<!\w)-(?!\w)', ' ', regex=True)

# PASO 3: Limpieza final de espacios sobrantes generados por los reemplazos
datos['descripcion_local'] = datos['descripcion_local'].str.strip()

print("\n✅ Texto Limpio (Listo para Tokenización):")
display(datos['descripcion_local'].iloc[3169][:150])
```



Puedes crear una expresión regular con la ayuda del sitio web [regex101.com](https://regex101.com/). Si quieres saber más sobre regex y su aplicación en bases de datos, vale la pena leer el artículo [Principales casos de uso de Regex para procesamiento de datos](https://www.alura.com.br/artigos/principais-casos-uso-regex-para-tratamento-dados), que muestra una aplicación regex en banco de datos.

In [30]:
datos['descripcion_local'] = datos['descripcion_local'].str.split()
datos.head()

,evaluacion_general,experiencia_local,max_hospedes,descripcion_local,descripcion_vecindad,cantidad_baños,cantidad_cuartos,cantidad_camas,modelo_cama,comodidades,cuota_deposito,cuota_limpieza,precio
0,10.0,--,1,"[this, clean, and, comfortable, one, bedroom, ...",Lower Queen Anne is near the Seattle Center (s...,1,1,1,Real Bed,"{Internet,""Wireless Internet"",Kitchen,""Free Pa...",0.0,0.0,110.0
1,10.0,--,1,"[our, century, old, upper, queen, anne, house,...","Upper Queen Anne is a really pleasant, unique ...",1,1,1,Futon,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",0.0,0.0,45.0
2,10.0,--,1,"[cozy, room, in, two-bedroom, apartment, along...",The convenience of being in Seattle but on the...,1,1,1,Futon,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",0.0,0.0,55.0
3,10.0,--,1,"[very, lovely, and, cozy, room, for, one, conv...","Ballard is lovely, vibrant and one of the most...",1,1,1,Pull-out Sofa,"{Internet,""Wireless Internet"",Kitchen,""Free Pa...",0.0,20.0,52.0
4,10.0,--,1,"[the, studio, at, mibbett, hollow', is, in, a,...",--,1,1,1,Real Bed,"{""Wireless Internet"",Kitchen,""Free Parking on ...",0.0,15.0,85.0


In [31]:
datos['comodidades'] = datos['comodidades'].str.replace('\{|}|\"','', regex=True)

<>:1: SyntaxWarning: invalid escape sequence '\{'
<>:1: SyntaxWarning: invalid escape sequence '\{'
/tmp/ipython-input-2646911477.py:1: SyntaxWarning: invalid escape sequence '\{'
  datos['comodidades'] = datos['comodidades'].str.replace('\{|}|\"','', regex=True)


In [32]:
datos['comodidades'] = datos['comodidades'].str.split(',')
datos.head()

,evaluacion_general,experiencia_local,max_hospedes,descripcion_local,descripcion_vecindad,cantidad_baños,cantidad_cuartos,cantidad_camas,modelo_cama,comodidades,cuota_deposito,cuota_limpieza,precio
0,10.0,--,1,"[this, clean, and, comfortable, one, bedroom, ...",Lower Queen Anne is near the Seattle Center (s...,1,1,1,Real Bed,"[Internet, Wireless Internet, Kitchen, Free Pa...",0.0,0.0,110.0
1,10.0,--,1,"[our, century, old, upper, queen, anne, house,...","Upper Queen Anne is a really pleasant, unique ...",1,1,1,Futon,"[TV, Internet, Wireless Internet, Kitchen, Fre...",0.0,0.0,45.0
2,10.0,--,1,"[cozy, room, in, two-bedroom, apartment, along...",The convenience of being in Seattle but on the...,1,1,1,Futon,"[TV, Internet, Wireless Internet, Kitchen, Fre...",0.0,0.0,55.0
3,10.0,--,1,"[very, lovely, and, cozy, room, for, one, conv...","Ballard is lovely, vibrant and one of the most...",1,1,1,Pull-out Sofa,"[Internet, Wireless Internet, Kitchen, Free Pa...",0.0,20.0,52.0
4,10.0,--,1,"[the, studio, at, mibbett, hollow', is, in, a,...",--,1,1,1,Real Bed,"[Wireless Internet, Kitchen, Free Parking on P...",0.0,15.0,85.0


### ✂️ Tokenización Estructural: El Método `.str.split()`

**El Concepto (De String a Lista):**
La tokenización convierte una cadena de texto continua (String) en una lista de Python `[...]`, donde cada elemento es un "Token" (generalmente una palabra). Esto permite contar frecuencias (ej. ¿Cuántas veces aparece la palabra "Wifi" en los anuncios de Airbnb?).

**La Solución Vectorizada:**
El método `.str.split(patrón_separador)` recorre toda la columna y corta el texto cada vez que encuentra el patrón indicado.
* **Separador por Defecto (Espacios):** Al usar `.str.split()` sin argumentos, Pandas asume que el separador es el espacio en blanco. Es ideal para tokenizar oraciones naturales (ej. "Casa hermosa" -> `['Casa', 'hermosa']`).
* **Separador Específico (Comas):** Cuando los datos vienen como una enumeración (ej. "TV,Wifi,Cocina"), indicamos explícitamente el separador `.str.split(',')` para que no corte palabras compuestas separadas por espacios.

**Flujo de Trabajo (Columna `comodidades`):**
1. **Limpieza Regex:** `.str.replace(r'\{|\}|\"', '', regex=True)` destruye llaves y comillas usando el operador lógico OR (`|`).

    * (Nota extra de Clean Code: se agrego la letra r antes de las comillas. En Python, eso significa "Raw String" (Cadena Cruda), y es la mejor práctica siempre que escribas un Regex para evitar que Python confunda los backslash \ con comandos propios del sistema).
2. **Tokenización:** `.str.split(',')` convierte el string resultante en una lista de Python real.





### 🏗️ Refactorización Senior: Dos Paradigmas de Limpieza Textual (NLP)

**Paradigma 1: Encadenamiento de Métodos (Method Chaining)**
Ideal para transformaciones lineales y específicas de una sola columna. Al envolver el código entre paréntesis `()`, le indicamos a Python que la instrucción continúa en las siguientes líneas. Esto elimina la necesidad de repetir el nombre de la variable (`datos['columna'] = ...`) múltiples veces, reduciendo el ruido visual, respetando el principio DRY y optimizando el uso de la memoria RAM.

**Paradigma 2: Modularización Vectorizada (Pipeline)**
Ideal para sistemas escalables. Creamos una función (Fábrica) que recibe una Serie completa de Pandas y le aplica las transformaciones vectorizadas. Si mañana necesitamos limpiar 10 columnas distintas, simplemente reutilizamos la función, centralizando el mantenimiento del código en un solo lugar.

In [33]:
# ==============================================================================
# PARADIGMA 1: METHOD CHAINING (Aplicado a 'descripcion_vecindad')
# ==============================================================================

# Al usar paréntesis externos, podemos tabular cada método de forma ordenada
datos['descripcion_vecindad'] = (
    datos['descripcion_vecindad']
    .str.lower()
    .str.replace(r'[^a-zA-Z0-9\-\']', ' ', regex=True)
    .str.replace(r'(?<!\w)-(?!\w)', '', regex=True)
    .str.split()
)

print("✅ 'descripcion_vecindad' limpiada y tokenizada usando Method Chaining.")

✅ 'descripcion_vecindad' limpiada y tokenizada usando Method Chaining.


In [43]:
# ==============================================================================
# PARADIGMA 2: MODULARIZACIÓN BLINDADA (Defensive Programming)
# ==============================================================================

# 1. Definimos el "Módulo" central de limpieza (Business Logic)
def limpiar_y_tokenizar_serie(serie_texto):
    """
    Recibe una Serie de Pandas (texto), la normaliza a minúsculas,
    rellena nulos para evitar colapsos, fuerza el tipo String,
    elimina caracteres especiales usando Regex, y la tokeniza.
    """
    serie_limpia = (
        serie_texto
        .fillna('')   # BLINDAJE 1: Transforma los nulos (NaN) en texto vacío
        .astype(str)  # BLINDAJE 2: Fuerza a Pandas a tratar toda la columna como texto
        .str.lower()
        .str.replace(r'[^a-zA-Z0-9\-\']', ' ', regex=True)
        .str.replace(r'(?<!\w)-(?!\w)', '', regex=True)
        .str.split()
    )
    return serie_limpia

# 2. Aplicamos la función a nuestra columna (o a cualquier otra en el futuro)
datos['experiencia_local'] = limpiar_y_tokenizar_serie(datos['experiencia_local'])

print("✅ 'descripcion_local' limpiada y tokenizada usando Modularización Vectorizada.")

# ==============================================================================
# AUDITORÍA FINAL
# ==============================================================================
display(datos[['descripcion_vecindad', 'experiencia_local']].head(3))

✅ 'descripcion_local' limpiada y tokenizada usando Modularización Vectorizada.


,descripcion_vecindad,experiencia_local
0,"[lower, queen, anne, is, near, the, seattle, c...",[]
1,"[upper, queen, anne, is, a, really, pleasant, ...",[]
2,"[the, convenience, of, being, in, seattle, but...",[]


#**4. Datos de tiempo**


---




### ⏱️ Cronología de Datos: La Biblioteca Nativa `datetime`

**El Valor de Negocio (Aritmética Temporal):**
En cualquier sistema transaccional, el tiempo es una magnitud matemática, no un texto. La biblioteca estándar `datetime` de Python nos permite calcular diferencias de días (para calcular intereses por mora) o proyectar fechas futuras (para establecer vencimientos).

**Clases Arquitectónicas Principales:**
1. **`datetime.date`**: Estructura que maneja únicamente la fecha (Año, Mes, Día). Ideal para fechas de nacimiento, feriados o vencimientos de facturas.
   * *Método clave:* `datetime.date.today()` (Obtiene la fecha actual del sistema operativo).
2. **`datetime.datetime`**: Estructura completa de Fecha + Hora (Año, Mes, Día, Hora, Minutos, Segundos, Microsegundos). Crucial para *Logs* de auditoría de software o *Timestamps* de transacciones bancarias.
   * *Método clave:* `datetime.datetime.now()` (Obtiene la marca de tiempo exacta del instante de ejecución).
3. **Operaciones (`timedelta`):** Python permite operaciones algebraicas directas. Al restar dos objetos de fecha (`fecha_b - fecha_a`), el motor devuelve automáticamente la diferencia exacta en días y segundos.

**📚 Material de Referencia (Documentación y Profundización):**
* [Documentación Oficial de Python: Biblioteca `datetime`](https://docs.python.org/3/library/datetime.html)
* [Artículo: Python datetime - ¿Cómo configuro la fecha y la hora en Python?](https://www.alura.com.br/artigos/lidando-com-datas-e-horarios-no-python)

In [35]:
dt_data = pd.read_json('/content/drive/MyDrive/Pandas/inmuebles_disponibles.json')
dt_data.head()

,id,fecha,lugar_disponible,precio
0,857,2016-01-04,False,None
1,857,2016-01-05,False,None
2,857,2016-01-06,False,None
3,857,2016-01-07,False,None
4,857,2016-01-08,False,None


In [36]:
dt_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 365000 entries, 0 to 364999
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                365000 non-null  int64 
 1   fecha             365000 non-null  object
 2   lugar_disponible  365000 non-null  bool  
 3   precio            270547 non-null  object
dtypes: bool(1), int64(1), object(2)
memory usage: 11.5+ MB


In [37]:
dt_data['fecha'] = pd.to_datetime(dt_data['fecha'])

### 🗓️ Ingeniería Temporal: Conversión a `datetime64`

**El Problema del Texto Crudo:**
Al importar archivos JSON o CSV, Pandas clasifica las fechas sistemáticamente como `object` (cadenas de texto). En este estado, es imposible realizar filtros cronológicos (ej. "filtrar ventas del último trimestre") o extraer componentes (día, mes, año).

**La Solución Vectorizada (`pd.to_datetime`):**
A diferencia de los números estándar donde usamos `.astype()`, el motor temporal requiere una función analítica dedicada. `pd.to_datetime()` parsea el texto y lo convierte al tipo de dato `datetime64[ns]` (precisión de nanosegundos).

**Impacto Analítico:**
Una vez que la columna es formalmente una serie de tiempo, "desbloqueamos" el atributo `.dt` de Pandas, el cual nos permite extraer algorítmicamente el día de la semana, el número de mes, o calcular deltas de tiempo (días transcurridos entre dos fechas) sin necesidad de procesar el texto manualmente.

In [38]:
dt_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 365000 entries, 0 to 364999
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                365000 non-null  int64         
 1   fecha             365000 non-null  datetime64[ns]
 2   lugar_disponible  365000 non-null  bool          
 3   precio            270547 non-null  object        
dtypes: bool(1), datetime64[ns](1), int64(1), object(1)
memory usage: 11.5+ MB


In [39]:
dt_data.sample(10)

,id,fecha,lugar_disponible,precio
147340,1188,2016-09-05,False,None
88379,1520,2016-02-22,True,$59.00
108061,3626,2016-01-25,False,None
263315,2748,2016-06-02,False,None
144102,701,2016-10-22,False,None
310371,878,2016-05-04,True,$85.00
131625,2453,2016-08-16,True,$411.00
334176,766,2016-07-23,True,$200.00
309034,3103,2016-09-04,True,$90.00
189086,3354,2016-01-20,False,None


In [40]:
dt_data['fecha'].dt.strftime('%Y-%m')

,fecha
0,2016-01
1,2016-01
2,2016-01
3,2016-01
4,2016-01
...,...
364995,2016-12
364996,2016-12
364997,2016-12
364998,2017-01


In [41]:
subset = dt_data.groupby(dt_data['fecha'].dt.strftime('%Y-%m'))['lugar_disponible'].sum()
subset

,lugar_disponible
fecha,
2016-01,16543
2016-02,20128
2016-03,23357
2016-04,22597
2016-05,23842
2016-06,23651
2016-07,22329
2016-08,22529
2016-09,22471


### 📈 Inteligencia de Negocios: Agrupación Temporal (`groupby` + `.dt`)

**El Valor Gerencial (Análisis de Estacionalidad):**
Las series de tiempo con datos diarios (ej. 365.000 registros) son imposibles de analizar visualmente. Para encontrar tendencias, necesitamos "enrollar" (Roll-up) los datos hacia una granularidad mayor, como meses o años.

**El Motor Analítico: El atributo `.dt.strftime()`**
* El atributo `.dt` "despierta" las funciones de tiempo de la columna.
* El método `strftime` (String Format Time) extrae componentes específicos y los formatea como texto.
* El patrón `'%Y-%m'` le indica a Pandas: *"Extrae el Año con 4 dígitos (`%Y`), pon un guion (`-`), y extrae el Mes con 2 dígitos (`%m`)"*.

**El Motor de Agrupación: `groupby()`**
Agrupa todas las filas que comparten el mismo mes y año en un solo bloque. Es el equivalente exacto a una Tabla Dinámica (Pivot Table) en Excel.

**El Truco Matemático (Suma de Booleanos):**
Al aplicar `.sum()` sobre una columna Booleana (`True`/`False`), Pandas procesa cada `True` como un `1` y cada `False` como un `0`. El resultado matemático es el conteo exacto de eventos positivos (lugares disponibles) en cada período agrupado.

# Desafío: hazlo tú mismo



En esta clase, aprendimos cómo manipular datos temporales usando datetime. Entendimos cómo transformar una columna a fecha y hora y luego manipular estos datos. Aun así, no todos los datos del conjunto inmuebles_disponibles.json fueron tratados.

Durante las clases de este curso, descubrimos cómo transformar y trabajar con valores numéricos, por ejemplo, eliminando valores numéricos dentro de un texto y transformándolos en un tipo numérico, como int64 o float64.

Sabiendo esto, en esta actividad te propongo transformar los datos de la columna precio del conjunto de datos inmuebles_disponibles.json al tipo numérico float64. Recordando que, antes de hacer esto, debes llenar los valores vacíos de la columna con un valor. Una sugerencia: reemplazar con el string '0.0'.

No dudes en seguir los mismos pasos dados en clase o, si lo prefieres, realizar otras mejoras, como eliminar algunos caracteres o palabras vacías. En el apartado “Opinión del instructor” encontrarás una posible resolución para esta actividad.

In [45]:
# importamos la biblioteca numpy
import numpy as np

# utilizamos el método fillna para llenar los elementos vacíos por '0.0'
# definimos el parámetro de inplace para True para substituir en el DataFrame
dt_data['precio'].fillna('0.0', inplace = True)

# borramos el $ y las comas con apply lambda
dt_data['precio'] = dt_data['precio'].apply(lambda x: x.replace('$', '').replace(',',''))

# transformamos los tipos de datos para float64
dt_data['precio'] = dt_data['precio'].astype(np.float64)

# observamos el resultado final
dt_data

/tmp/ipython-input-2692693818.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dt_data['precio'].fillna('0.0', inplace = True)


,id,fecha,lugar_disponible,precio
0,857,2016-01-04,False,0.0
1,857,2016-01-05,False,0.0
2,857,2016-01-06,False,0.0
3,857,2016-01-07,False,0.0
4,857,2016-01-08,False,0.0
...,...,...,...,...
364995,3279,2016-12-29,True,140.0
364996,3279,2016-12-30,True,140.0
364997,3279,2016-12-31,True,140.0
364998,3279,2017-01-01,True,140.0


### 🇦🇷 Sanitización Financiera: Localización Argentina (ARS)

**El Desafío de Negocio:**
Los sistemas y bancos en Argentina exportan los valores financieros utilizando el punto (`.`) para separar miles y la coma (`,`) para los decimales (Ej: `$1.250,50`). Python y NumPy utilizan el estándar internacional inverso (Ej: `1250.50`).

**El Algoritmo de Conversión (El orden es estricto):**
1. `.str.replace('$', '')`: Eliminar símbolo de moneda.
2. `.str.replace('.', '')`: Eliminar separadores de miles para agrupar el entero.
3. `.str.replace(',', '.')`: Reemplazar la coma decimal por el punto de punto flotante de Python.
4. `.astype(np.float64)`: Conversión final a formato matemático.

In [46]:
import pandas as pd
import numpy as np

# 1. Simulamos una extracción de datos de ventas de tu PyME (Formato AR)
df_pyme = pd.DataFrame({
    'factura_id': [1001, 1002, 1003],
    'total_facturado': ['$1.250,50', '$5.000,00', '$10.450,99']
})

print("❌ Datos Crudos (Texto con Localización Argentina):")
display(df_pyme)

# 2. Pipeline de Sanitización Financiera (Method Chaining)
df_pyme['total_facturado'] = (
    df_pyme['total_facturado']
    .str.replace('$', '', regex=False)   # Paso 1: Chau símbolo
    .str.replace('.', '', regex=False)   # Paso 2: Chau separador de miles
    .str.replace(',', '.', regex=False)  # Paso 3: Coma a Punto (Traducción a Python)
    .astype(np.float64)                  # Paso 4: Casting matemático
)

# 3. Auditoría Final
print("\n✅ Datos Limpios (NumPy Float64 listos para operaciones):")
display(df_pyme)
print("\nTipos de Datos:")
print(df_pyme.dtypes)

❌ Datos Crudos (Texto con Localización Argentina):


,factura_id,total_facturado
0,1001,"$1.250,50"
1,1002,"$5.000,00"
2,1003,"$10.450,99"



✅ Datos Limpios (NumPy Float64 listos para operaciones):


,factura_id,total_facturado
0,1001,1250.50
1,1002,5000.00
2,1003,10450.99



Tipos de Datos:
factura_id           int64
total_facturado    float64
dtype: object
